In [1]:
# Standard Library Imports
import os
import io
import json

# Third-Party Library Imports
import numpy as np
import matplotlib.pyplot as plt
import fitz  # PyMuPDF
from PIL import Image
import torch
import camelot.io as camelot
import camelot.plotting as cpl
import IPython.display as display

# LangChain and Vector Store Libraries
from langchain.vectorstores import Qdrant
from langchain.retrievers import MergerRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

# Transformers for Models
from transformers import CLIPProcessor, CLIPModel
from transformers import (
    Qwen2VLForConditionalGeneration,
    AutoTokenizer,
    AutoProcessor,
)
from qwen_vl_utils import process_vision_info

# Qdrant Client and Models
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

c:\Users\kenil\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\retrievers\document_compressors\chain_extract.py:13: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain.chains.llm import LLMChain
c:\Users\kenil\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_internal\_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `ResponseSchema` to V2.
  warn(
c:\Users\keni

In [22]:
# Initializing Models

# Text embedding model
text_embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-base-en-v1.5"
)
# Image embedding model
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Qwen Model
qwen_model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", torch_dtype="auto"
)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
qwen_model.to(DEVICE)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen2VLForConditionalGeneration(
  (visual): Qwen2VisionTransformerPretrainedModel(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2VLVisionBlock(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): VisionSdpaAttention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): VisionMlp(
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (act): QuickGELUActivation()
          (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        )
      )
    )
    (merger): PatchMerger(
      (ln_q): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): Seq

#### Reading the Main Document

##### Extracting Tables from the Document

In [ ]:
tables = camelot.read_pdf(
    "medical_information_pdf.pdf", pages="19-end", flavor="lattice"
)

In [ ]:
tables[0].df

In [ ]:
# Process tables
all_tables_data = []  # List to store processed tables

for i, table in enumerate(tables):
    headers = table.df.iloc[0].tolist()  # Extract headers
    data = table.df.iloc[1:].values.tolist()  # Extract table data
    table_data = [
        dict(zip(headers, row)) for row in data
    ]  # Convert to list of dicts
    all_tables_data.append(
        {"table_id": i + 1, "data": table_data}
    )  # Store with table ID

# Save to JSON file
with open("tables.json", "w", encoding="utf-8") as f:
    json.dump(all_tables_data, f, indent=4, ensure_ascii=False)

print("✅ All tables extracted and saved in tables.json")

In [ ]:
import json

with open("tables.json", "r", encoding="utf-8") as file:
    data = json.load(file)
for indices in data:
    for key, value in indices.items():
        print(f"{key}: {value}")

In [ ]:
data

In [ ]:
cpl.prepare_plot(tables[7])
plt.show()

### Extracting PDF content using Fitz and camelot - CODE STARTS

#### PDF Input

In [ ]:
from content_extract import pdf_content_extraction

content = pdf_content_extraction("pdf.pdf")

### Text Overlap to maintain semantic relationship

In [ ]:
from embeddings import split_text

split_texts = split_text(content["text"])
print("Number of text chunks:", len(split_texts))

Number of text chunks: 39


### Generate Image Embeddings


In [ ]:
from embeddings import image_generate_embeddings

image_embeddings, px, size = image_generate_embeddings(
    content["images"], clip_processor, clip_model
)

### Generate Table Embeddings

In [ ]:
from embeddings import generate_table_embeddings

table_embeddings = generate_table_embeddings(content, text_embedding_model)

print("✅ Generated embeddings for", len(table_embeddings), "tables!")

✅ Generated embeddings for 3 tables!


#### Creating Vector Database Using Qdrant for storing Text, Image and Table Embeddings Separately

In [12]:
TEXT_COLLECTION = "text_collection"
IMAGE_COLLECTION = "image_collection"
TABLE_COLLECTION = "table_collection"

##### Storing Text Embeddings

In [ ]:
from store_embeddings import store_text

text_store = store_text(split_texts, text_embedding_model, TEXT_COLLECTION)
print("✅ Text embeddings stored in Qdrant!")

✅ Text embeddings stored in Qdrant!


#### Create Embedding Collection

In [14]:
client = QdrantClient(":memory:")  # Create an in-memory Qdrant client

##### Storing Image Embeddings

In [ ]:
from store_embeddings import store_image_embeddings

IMAGE_VECTOR_SIZE = 512
store_image_embeddings(
    client,
    content["images"],
    image_embeddings,
    px,
    size,
    IMAGE_COLLECTION,
    IMAGE_VECTOR_SIZE,
)

Collection 'image_collection' created successfully!
Stored 22 images in Qdrant.


##### Storing Table Embeddings

In [ ]:
from store_embeddings import store_table_embeddings

TABLE_VECTOR_SIZE = 768  # adjust as per your model
store_table_embeddings(
    client, TABLE_COLLECTION, TABLE_VECTOR_SIZE, table_embeddings
)

Collection 'table_collection' created successfully!
✅ Table embeddings stored in Qdrant successfully!


#### Retrieving Relevant Information

In [17]:
query = "what are the parts of asmanex hfa?"

##### From Text Embeddings

In [18]:
retriever = text_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7},
)

##### From Image Embeddings

In [19]:
def image_retrieval(
    query,
    clip_processor,
    clip_model,
    collection_name,
    limit=3,
    with_payload=True,
    score_threshold=0.7,
):
    """
    Retrieves images based on a text query using CLIP model embeddings.

    This function takes a text query, generates its embedding using the CLIP model,
    and performs a similarity search in a specified Qdrant collection. It returns
    the top matching images based on the query embedding and displays them.

    Args:
        query (str): The text query used to search for relevant images.
        clip_processor (CLIPProcessor): The processor used to tokenize and preprocess input text for CLIP.
        clip_model (CLIPModel): The CLIP model used to generate text embeddings.
        collection_name (str): The name of the Qdrant collection containing the image embeddings.
        limit (int, optional): The maximum number of results to return (default is 3).
        with_payload (bool, optional): Whether to include metadata (e.g., filenames) with the results (default is True).
        score_threshold (float, optional): The minimum similarity score for returning results (default is 0.7).

    Returns:
        None: Displays the top matching images based on the query
    """
    input_text = clip_processor(text=[query], return_tensors="pt")
    with torch.no_grad():
        text_embedding = (
            clip_model.get_text_features(**input_text).squeeze().tolist()
        )
    results_with_scores = client.search(
        collection_name=collection_name,
        query_vector=text_embedding,
        limit=limit,
        with_payload=with_payload,  # Retrieve metadata (e.g., filenames)
        score_threshold=score_threshold,
    )
    results = [res.payload["filename"] for res in results_with_scores]
    if results:
        for image_path in results:
            img = Image.open(image_path)
            display.display(img)
    return results


image_results = image_retrieval(
    query, clip_processor, clip_model, IMAGE_COLLECTION
)

##### From Table Embeddings

In [20]:
table_retriever = Qdrant(
    client=client,
    collection_name=TABLE_COLLECTION,
    embeddings=text_embedding_model,
    content_payload_key="table_text",
).as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7},
)

C:\Users\kenil\AppData\Local\Temp\ipykernel_17488\536263116.py:1: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 0.3.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  table_retriever = Qdrant(


#### Re-Ranking the results

In [21]:
merger_retriever = MergerRetriever(
    retrievers=[retriever, table_retriever],
    # weights=[0.5, 0.5],  # Adjust weights to balance retrieval sources
)
retrieved_docs = merger_retriever.get_relevant_documents(query)
output = " ".join([doc.page_content for doc in retrieved_docs])
print("Merged output:", output)
# for doc in retrieved_docs:
#     print(doc.page_content)
#     print(doc.metadata)

Merged output: The parts of your ASMANEX HFA:
There are 2 main parts to your ASMANEX HFA inhaler: the metal canister that holds the medicine and 
the blue plastic actuator that sprays the medicine from the canister. 

The inhaler also has a pink cap that covers the mouthpiece of the actuator (see Figure 1). The 
cap from the mouthpiece must be removed before use. The inhaler contains “120” actuations 
(puffs).
Figure 1 

The inhaler comes with a dose counter located on the plastic actuator (see Figure 1). The General Information about the safe and effective use of ASMANEX HFA.
Medicines are sometimes prescribed for purposes other than those listed in a Patient Information leaflet. Do not use 
ASMANEX HFA for a condition for which it was not prescribed. Do not give your ASMANEX HFA to other people, even if 
they have the same condition that you have. It may harm them.
This Patient Information leaflet summarizes the most important information about ASMANEX HFA. If you would like more 


C:\Users\kenil\AppData\Local\Temp\ipykernel_17488\1775841980.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = merger_retriever.get_relevant_documents(query)


### Integrating Multimodal LLM

In [25]:
qwen_processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [26]:
qwen_model.eval()

Qwen2VLForConditionalGeneration(
  (visual): Qwen2VisionTransformerPretrainedModel(
    (patch_embed): PatchEmbed(
      (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
    )
    (rotary_pos_emb): VisionRotaryEmbedding()
    (blocks): ModuleList(
      (0-31): 32 x Qwen2VLVisionBlock(
        (norm1): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (norm2): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
        (attn): VisionSdpaAttention(
          (qkv): Linear(in_features=1280, out_features=3840, bias=True)
          (proj): Linear(in_features=1280, out_features=1280, bias=True)
        )
        (mlp): VisionMlp(
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (act): QuickGELUActivation()
          (fc2): Linear(in_features=5120, out_features=1280, bias=True)
        )
      )
    )
    (merger): PatchMerger(
      (ln_q): LayerNorm((1280,), eps=1e-06, elementwise_affine=True)
      (mlp): Seq

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate(
    input_variables=["query", "output"],
    template=(
        "Given the following medical query and its context, as well as a relevant image, craft a well written and easy to understand Answer by extracting relevant information from the context.\n\n"
        "Query:\n{query}\n\n"
        "Context:\n{output}\n\n"
        "Refer to the image if helpful.\n\n"
        "Answer:"
    ),
)

In [28]:
if image_results:
    images = [Image.open(path) for path in image_results]

In [55]:
def generate_response(query, output_text, image_path=None):
    """
    Generate a response using the Qwen2VL model with text and image input.

    Args:
        query (str): The user's query/question.
        output_text (str): The relevant context or data retrieved (text and table).
        image_path (str, optional): Path to the image for multimodal input.

    Returns:
        str: The generated response.
    """
    # Format the prompt using LangChain template
    prompt = prompt_template.format(query=query, output=output_text)

    # Load image if provided
    if image_path:
        image = Image.open(image_path)
    else:
        image = None

    # Prepare the inputs for the model
    inputs = qwen_processor(text=prompt, images=image, return_tensors="pt").to(
        DEVICE
    )

    # Generate the output from the model
    with torch.no_grad():
        generated_ids = qwen_model.generate(**inputs, max_new_tokens=512)

    # Decode the generated tokens to text
    response = qwen_processor.decode(
        generated_ids[0], skip_special_tokens=True
    )
    summary_start = response.find("Answer:") + len("Answer:")
    summary = response[summary_start:].strip()

    return summary


if image_results:
    response = generate_response(query, output, image_path=image_results)
else:
    response = generate_response(query, output)
print("Generated Response:")
print(response)

Generated Response:
The parts of your ASMANEX HFA include the metal canister that holds the medicine, the blue plastic actuator that sprays the medicine from the canister, the pink cap that covers the mouthpiece of the actuator, and the dose counter located on the plastic actuator. The inhaler also contains "120" actuations (puffs). The inhaler comes with a dose counter and a pink cap that must be removed before use. The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler contains "120" actuations (puffs). The inhaler con

In [ ]:
from huggingface_hub import InferenceClient
import json

model_id = "Qwen/Qwen2-VL-2B-Instruct"
qwen_model = InferenceClient(
    model=model_id, token=huggingface_token, timeout=60
)

In [ ]:
import torch


print("CUDA available:", torch.cuda.is_available())



print(
    "Device name:",
    torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
)

In [ ]:
import torch


print("CUDA available:", torch.cuda.is_available())


print("Device count:", torch.cuda.device_count())


print(
    "Device name:",
    torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU",
)